In [2]:

import numpy as np

import model
from model import MyDataSet

import torch
import torch.utils.data as Data

from model import MyDataSet,generate_process,process2
import model
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2

# 生成和测试

In [2]:

save_path3   = 'result/last_result_16/'
pic_path     = 'result/test_pic/'

noise_dim  = 16
mesh2num   = 16
BATCH_SIZE = 128
batch_size = BATCH_SIZE
ffd_dim    = (3,7,1)
point_num  = (ffd_dim[0]+1)*(ffd_dim[1]+1)*(ffd_dim[2]+1)
dim_mesh   = (18,68,3)
upper_vdim = 64
lower_vdim = 11
device     = 'cpu'
mesh2num_model = model.ConvNet(mesh2num).to(device) 
G = model.DeepConvNet3(noise_dim + mesh2num,point_num,upper_vdim).to(device)
D = model.DeepConvNet2(batch_size,1).to(device) #输出为1维向量
    
mesh2num_model.load_state_dict(torch.load(save_path3 +"m2n_param.pkl"))
G.load_state_dict(torch.load(save_path3 +"generator_param.pkl"))
D.load_state_dict(torch.load(save_path3 +"discriminator_param.pkl"))


<All keys matched successfully>

In [3]:
data_root   = 'F:\\graduate_student\\T2_GANpropeller\\test2\\1_model\\grid_mesh\\'

base_meshs = np.load(data_root + 'mesh_data_test.npy')


In [6]:
mean_mesh = np.mean(base_meshs,0)
base_mesh = torch.from_numpy(mean_mesh)

p1 = generate_process(batch_size=32, noise_size=noise_dim,mesh2num=mesh2num,upper=upper_vdim,device='cpu')

p2 = process2(p1,mesh2num_model,G,D)

fake_ffdpca = p2.sample(base_mesh,64,pic_path)
np.save(pic_path + 'complicated_ffdpca',fake_ffdpca)

base max c: 0.1990192041774946 max beta: 22.22549356306708
0 max c: 0.19385693190249775 max beta: 30.22238534138534
1 max c: 0.17711801413569855 max beta: 31.1187878165556
2 max c: 0.19628152364902304 max beta: 20.333006280361978
3 max c: 0.19901136772172442 max beta: 32.24126494416325
4 max c: 0.1886738114738666 max beta: 39.10379230941474
5 max c: 0.18728862898858525 max beta: 23.427760332384146
6 max c: 0.17831271891305217 max beta: 28.182481738860538
7 max c: 0.2086165552898738 max beta: 12.782266811398811
8 max c: 0.17106349129467002 max beta: 22.89514350672392
9 max c: 0.20227881212440466 max beta: 37.69963964931058
10 max c: 0.18599401269209412 max beta: 35.16065740216531
11 max c: 0.16354096616373012 max beta: 26.127748522390494
12 max c: 0.18620847261848444 max beta: 37.06149598019706
13 max c: 0.19392431978849228 max beta: 21.519300521089274
14 max c: 0.18908387845325453 max beta: 37.67894972606554
15 max c: 0.17236043183357913 max beta: 32.033048900971146
16 max c: 0.2003795

In [8]:
from pyDOE import lhs

BATCH_SIZE = 4452
p = 100
a = np.array(lhs(1,p) * BATCH_SIZE,dtype=int)[:,0]
base_plot = base_meshs[a]
np.save( pic_path+'base_meshs', base_plot)
min_batch = 10
base_plots = torch.from_numpy(base_plot)
fake_ffdpcas = []
for i in range(p):
    base_mesh = base_plots[i]
    p1 = generate_process(batch_size=min_batch, noise_size=noise_dim,mesh2num=mesh2num,upper=upper_vdim,device=device)
    p2 = process2(p1,mesh2num_model,G,D)
    fake_ffdpca = p2.sample(base_mesh,min_batch,pic_path,False)
    fake_ffdpcas.append(fake_ffdpca)

fake_ffdpcas = np.array(fake_ffdpcas)
np.save(pic_path + 'complicated_ffdpca',fake_ffdpcas)

# 测试连续生成

# easy的生成测试


In [15]:
save_path3   = 'result/last_result_easy/'
pic_path     = 'result/test_pic/'
noise_dim  = 8
mesh2num   = 8
BATCH_SIZE = 256
batch_size = BATCH_SIZE
ffd_dim    = (3,7,1)
point_num  = (ffd_dim[0]+1)*(ffd_dim[1]+1)*(ffd_dim[2]+1)
dim_mesh   = (18,68,3)
upper_vdim = 64
lower_vdim = 11
device = 'cpu'

mesh2num_model = model.ConvNet(mesh2num).to(device) 
G = model.DeepConvNet_easy2(noise_dim,point_num,upper_vdim).to(device)
D = model.DeepConvNet2(batch_size,1).to(device) #输出为1维向量

mesh2num_model.load_state_dict(torch.load(save_path3 +"m2n_param.pkl"))
G.load_state_dict(torch.load(save_path3 +"generator_param.pkl"))
D.load_state_dict(torch.load(save_path3 +"discriminator_param.pkl"))

<All keys matched successfully>

In [16]:
p1 = generate_process(batch_size=256, noise_size=noise_dim,mesh2num=mesh2num,upper=upper_vdim,device=device)

p2 = process2(p1,mesh2num_model,G,D)

In [29]:
data_root   = 'F:\\graduate_student\\T2_GANpropeller\\test5\\0_database\\mixed_model\\'

geom_list = np.load(data_root + 'easy_real.npy')
base_mesh = torch.from_numpy(np.mean(geom_list,0))


(3000, 18, 68, 3)


In [ ]:

easy_ffdpca =   p2.easy_sample(base_mesh,pic_path,False)

In [25]:
p = []
for i in easy_ffdpca:
    bottle = i[0,:,:]
    center = np.mean(bottle,0)
    p.append(i - center)
p = np.array(p)
np.save('result/test_pic/fake_mesh2',p)

In [26]:
a = np.load('result/test_pic/fake_mesh0.npy')
b = np.load('result/test_pic/fake_mesh1.npy')
c = np.load('result/test_pic/fake_mesh2.npy')

data  = np.concatenate((a,b,c),0)
print(data.shape)
np.save('result/test_pic/fake_mesh.npy',data)

(768, 18, 68, 3)


In [76]:
# 生成顺序[1,2,5,7,9],[8,16,32,64]

def get_easy_result(save_path3,noise_dim,save_path,batch_size,base_meshs):

    ffd_dim    = (3,7,1)
    point_num  = (ffd_dim[0]+1)*(ffd_dim[1]+1)*(ffd_dim[2]+1)
    dim_mesh   = (18,68,3)
    upper_vdim = 64
    lower_vdim = 11
    device = 'cpu'
    mesh2num = noise_dim
    mesh2num_model = model.ConvNet(mesh2num).to(device) 
    G = model.DeepConvNet_easy2(noise_dim+mesh2num,point_num,upper_vdim).to(device)
    D = model.DeepConvNet2(batch_size,1).to(device) #输出为1维向量

    mesh2num_model.load_state_dict(torch.load(save_path3 +"/m2n_param.pkl"))
    G.load_state_dict(torch.load(save_path3 +"/generator_param.pkl"))
    D.load_state_dict(torch.load(save_path3 +"/discriminator_param.pkl"))

    p1 = generate_process(batch_size=batch_size//base_meshs.shape[0], noise_size=noise_dim,mesh2num=mesh2num,upper=upper_vdim,device=device)

    p2 = process2(p1,mesh2num_model,G,D)
    easy_ffdpcas = []
    for base_mesh in base_meshs: 
        easy_ffdpca =   p2.easy_sample(base_mesh,pic_path,False)
        easy_ffdpcas.append(easy_ffdpca)
    
    easy_ffdpcas = np.array(easy_ffdpcas)
    easy_ffdpcas = easy_ffdpcas.reshape(-1,18,68,3)
    np.save(save_path,easy_ffdpcas)

In [64]:
name_list1 = ['base1','base2','base3','base5','base7','base9']
noise_num1  = [8,16,16,16,16,16]
name_list2 = ['easy8','easy16','easy32','easy64']
noise_num2  = [8,16,32,64]
base_list  = [ [1500],
                [700,2300],
                [500,1500,2500],
                [500,1200,1500,1700,2500],
                [300,700,1200,1500,1700,2300,2500],
                [300,500,700,1200,1500,1700,2300,2500,2700]]

def get_base_meshs(base_po,geom_list):
    
    base_meshs = []
    for i in base_po:
        base_meshs.append(geom_list[i])
    base_meshs = torch.from_numpy(np.array(base_meshs))
    return base_meshs

save_path_list1  = ['result/test_pic/'+_ for _ in name_list1]
save_path_list2  = ['result/test_pic/'+_ for _ in name_list2]

name_list1 = ['result/last_result_'+ _ for _ in name_list1]
name_list2 = ['result/last_result_'+ _ for _ in name_list2]

In [77]:
for i in range(len(name_list1)):
    name = name_list1[i]
    base_meshs = get_base_meshs(base_list[i],geom_list)
    get_easy_result(name_list1[i],noise_num1[i],save_path_list1[i],512,base_meshs)

In [82]:
for i in range(len(name_list2)):
    name = name_list2[i]
    print(i)
    base_meshs = get_base_meshs([500,1500,2500],geom_list)
    get_easy_result(name_list2[i],noise_num2[i],save_path_list2[i],512,base_meshs)

0
1
2
3


In [65]:
import random
def easy_surface(foil_data,ax, path = ''):
    # p = foil_data
    foil_data = foil_data.reshape(-1,3)
    x = foil_data[:,0]
    y = foil_data[:,1]
    z = foil_data[:,2]


    # ax.set_xlim(0,1)
    # ax.set_ylim(-0.5,0.5)
    # ax.set_zlim(-0.5,0.5)

    # ax.plot_surface(x,y,z,rstride = 1,cstride = 1, cmap= 'gist_yarg',alpha = 0.3)
    ax.scatter3D(x,y,z,s=0.5,alpha = 0.1,c='b')
    return ax

def get_meshcloud(new_meshs,save_path,num  = 100):

    posi =random.sample(range(1,new_meshs.shape[0]),num)
    new_meshs = new_meshs[posi]
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.zaxis.set_visible(False)
    for i in new_meshs:
        ax = easy_surface(i,ax)

    ax.view_init(elev=75, azim=0)
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

In [78]:
for i in save_path_list1:
    tmp = np.load(i + '.npy')
    print(tmp.shape)
    get_meshcloud(tmp,i + '.png')

(512, 18, 68, 3)
(512, 18, 68, 3)
(510, 18, 68, 3)
(510, 18, 68, 3)
(511, 18, 68, 3)
(504, 18, 68, 3)


In [83]:
for i in save_path_list2:
    tmp = np.load(i + '.npy')
    print(tmp.shape)
    get_meshcloud(tmp,i + '.png')

(510, 18, 68, 3)
(510, 18, 68, 3)
(510, 18, 68, 3)
(510, 18, 68, 3)


In [79]:
get_meshcloud(geom_list,'result/test_pic/real.png')